In [9]:
%%writefile question2d.py
import copy
from collections import namedtuple
from itertools import count
import math
import random
import numpy as np 
import time

import gym

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

from collections import namedtuple
import random
from PIL import Image
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

# parsing the arguments
import argparse as arg
arg_parser = arg.ArgumentParser(description='2b.py')
arg_parser.add_argument("--environment", type=str, help="env")
arg_parser.add_argument("--num_episodes", type=int, help="number of episodes")
arg_parser.add_argument("--batch", type=int, help="batch_size")
arg_parser.add_argument("--gamma", type=float, help="gamma")
arg_parser.add_argument("--learning_rate", type=float, help="learning rate")
arg_parser.add_argument("--mean_n", type=int, help="n-mean episodes")

# function for plot
def plot(rewards_mean, steps, best_rewards_mean, policy):
    # creating figure
    fig = plt.figure(figsize=(10,5)) 
    
    # First subplot showing performance
    ax1 = fig.add_subplot(121)
    ax1.plot(steps,rewards_mean,label=str(mean_n)+"-mean Reward")
    ax1.plot(steps,best_rewards_mean,label="Best Mean Reward")
    ax1.grid()
    ax1.set_xlabel("Number of steps")
    ax1.set_ylabel("Reward")
    ax1.legend()
    ax1.set_title("Performance of Agent")

    # Second subplot showing action choices of trained agent
    ax2 = fig.add_subplot(122)
    colour = {0:'lime',1:'red',2:'blue'}
    X = np.random.uniform(-1.8,0.9,1000)
    Y = np.random.uniform(-0.05,0.05,1000)
    Z = []
    for i in range(len(X)):
        temp = np.array([X[i],Y[i]])
        temp = torch.from_numpy(temp).float()
        temp = policy(temp)
        temp = torch.max(temp,dim=-1)[1]
        temp = temp.item()
        Z.append(temp)
    Z = pd.Series(Z)
    colour = Z.apply(lambda x:colour[x])
    plt.set_cmap('brg')
    ax2.scatter(X,Y,c=Z)
    ax2.set_title("Action Choices for trained Agent")
    ax2.set_xlabel("Position")
    ax2.set_ylabel("Velocity")
    action_choices = ['Left','Right','Nothing']
    fc_calc = sorted(colour.unique())
    legend_recs = [(mpatches.Rectangle((0,0),1,1,fc=fc_calc[i])) for i in range(3)]
    ax2.legend(legend_recs,action_choices,loc=4,ncol=3)

    plt.savefig(file_name)

# the DQN model
class DQN(nn.Module):
    def __init__(self):

        super(DQN, self).__init__()
        hidden_layer = 200
        input_layer, output_layer = 2, 3
        self.fc1 = nn.Linear(input_layer, hidden_layer,bias=False)
        self.fc2 = nn.Linear(hidden_layer, output_layer,bias=False)
        
    def forward(self, x):
        return self.fc2(self.fc1(x))

# Structure to store in the memory
Transition = namedtuple('Transion', ('state', 'action', 'next_state', 'reward'))

# Memory Unit
class ReplayMemory(object):
    def __init__(self, capacity):
        self.position = 0
        self.capacity = capacity
        self.memory = []
        
    def push(self, *args):
        if self.capacity > len(self.memory):
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position += 1
        self.position %= self.capacity
        
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    
def optimize_model():
    if len(memory.memory) < batch_size:
        return
    
    it = 0
    while it < 100:
        transitions = memory.sample(batch_size)
        transitions = zip(*transitions)
        batch = Transition(*transitions)
        
        rewards = tuple((map(lambda _reward: torch.tensor([_reward]), batch.reward))) 
        actions = tuple((map(lambda _action: torch.tensor([[_action]]), batch.action))) 

        mask_non_final = tuple(map(lambda s: s is not None, batch.next_state))
        mask_non_final = torch.tensor(mask_non_final, dtype=torch.bool)
        
        states_non_final = []
        for s in batch.next_state:
            if s is not None:
                states_non_final.append(s)
        states_non_final = torch.cat(states_non_final)
        states_non_final = states_non_final.to(device)

        batch_state, batch_action, batch_reward  = torch.cat(batch.state).to(device), torch.cat(actions), torch.cat(rewards)
        
        action_state_values = policy_network(batch_state)
        action_state_values = action_state_values.gather(1, batch_action)
        
        state_next_values = torch.zeros(batch_size)
        with torch.no_grad():
            state_next_actions = policy_network(states_non_final).max(1, keepdim=True)[1]
        state_next_values[mask_non_final] = target_network(states_non_final).gather(1, state_next_actions).squeeze(1).detach()

        action_state_values_expectation = (state_next_values * gamma) + batch_reward
        action_state_values_expectation = action_state_values_expectation.unsqueeze(1)
        action_state_values_expectation = action_state_values_expectation.float()
        
        loss = F.mse_loss(action_state_values, action_state_values_expectation)
        
        optimizer.zero_grad()
        loss.backward()
        for param in policy_network.parameters():
            param.grad.data.clamp_(-1, 1)
        optimizer.step()
        it += 1

# select the action
def select_action(state):
    global total_steps
    total_steps += 1
    rand_value = random.random()
    episode_threshold = episode_end + (episode_start - episode_end)*math.exp(-1.0 * total_steps/episode_decay)
    if rand_value > episode_threshold:
        with torch.no_grad():
            network = policy_network(state.to(device))
            network = network.max(1)[1]
            network = network.view(1,1)
            return network
    else:
        rand_range = random.randrange(3)
        ret = torch.tensor([[rand_range]],device=device,dtype=torch.long)
    return ret

# fetch the current state
def get_state(obs):
    state = np.array(obs)
    state = torch.from_numpy(state)
    state = state.type(torch.FloatTensor)
    state = state.unsqueeze(0)
    return state

# training in the environment
def train(env):
    episode_rewards = []
    best_reward = -200.0
    success = 0
    rewards_mean = []
    best_rewards_mean = []
    steps = []
    episode = 0
    while episode < num_episodes:
        obs = env.reset()
        state = get_state(obs)
        total_reward = 0.0
        for t in count():
            action = select_action(state)
            action = action.item()
            a = env.step(action)
            obs = a[0]
            reward = a[1]
            done = a[2]
            recall=[]
            total_reward += reward
            if 0.5 <= obs[0]:
                reward = obs[0] + 1.5
            else:
                reward = obs[0] + 0.5
            reward = torch.tensor([reward],device=device)
            if done:
                next_state = None
            else:
                next_state = get_state(obs)
            memory.push(state,action,next_state,reward.to('cpu'))
            state = next_state

            if total_steps > initial_memory:
                optimize_model()
            if done:
                if obs[0]>=0.5:
                    success+=1
                break
            if episode % mean_n == 1:
                target_network.load_state_dict(policy_network.state_dict())
        episode_rewards.append(total_reward)
        if episode >= mean_n:
            present_mean = np.mean(episode_rewards[-mean_n:])
            rewards_mean.append(present_mean)
            best_reward = max(present_mean,best_reward)
            best_rewards_mean.append(best_reward)
            steps.append(total_steps)
        print("Step Number: {} \t Episode Number: {} \t Reward: {} \t Success: {}".format(total_steps,episode,total_reward,success))
        episode+=1
    d = {'steps':steps, 'rewards_mean': rewards_mean, 'best_rewards_mean': best_rewards_mean}
    np.save('results.npy',d)
    plot(rewards_mean,steps,best_rewards_mean,policy_network)
    env.close()
    return

# setup device 
device = torch.device("cpu")

# extract the given arguments
arguments = arg_parser.parse_args()

# Set the Hyperparameters
batch_size = arguments.batch
gamma = arguments.gamma
num_episodes = arguments.num_episodes
learning_rate = arguments.learning_rate
mean_n = arguments.mean_n
env = gym.make(arguments.environment)

# other parametets
episode_start = 1.0
episode_end = 0.02
episode_decay = 1000
initial_memory = 1000

# creating file name to save the plot
file_name=arguments.environment+'_'+str(num_episodes)+'_'+str(batch_size)+'_ddqn.png'

# create newtorks
policy_network = DQN().float().to(device)
target_network = DQN().float().to(device)
target_network.load_state_dict(policy_network.state_dict())

# setup optimizer
optimizer = optim.Adam(policy_network.parameters(),lr=learning_rate)
total_steps = 0
memory = ReplayMemory(initial_memory)

# training in the environment
train(env)

Overwriting question2d.py


In [ ]:
!python3 question2d.py --environment="MountainCar-v0" --batch=8 --gamma=0.9 --num_episodes=100 --learning_rate=1e-4 --mean_n=5

Step Number: 200 	 Episode Number: 0 	 Reward: -200.0 	 Success: 0
Step Number: 400 	 Episode Number: 1 	 Reward: -200.0 	 Success: 0
Step Number: 600 	 Episode Number: 2 	 Reward: -200.0 	 Success: 0
Step Number: 800 	 Episode Number: 3 	 Reward: -200.0 	 Success: 0
Step Number: 1000 	 Episode Number: 4 	 Reward: -200.0 	 Success: 0
Step Number: 1200 	 Episode Number: 5 	 Reward: -200.0 	 Success: 0
Step Number: 1400 	 Episode Number: 6 	 Reward: -200.0 	 Success: 0
Step Number: 1600 	 Episode Number: 7 	 Reward: -200.0 	 Success: 0
Step Number: 1800 	 Episode Number: 8 	 Reward: -200.0 	 Success: 0
Step Number: 2000 	 Episode Number: 9 	 Reward: -200.0 	 Success: 0
Step Number: 2200 	 Episode Number: 10 	 Reward: -200.0 	 Success: 0
Step Number: 2400 	 Episode Number: 11 	 Reward: -200.0 	 Success: 0
Step Number: 2600 	 Episode Number: 12 	 Reward: -200.0 	 Success: 0
Step Number: 2800 	 Episode Number: 13 	 Reward: -200.0 	 Success: 0
Step Number: 3000 	 Episode Number: 14 	 Reward: